In [15]:
from ngsolve import *
from netgen.occ import *
from ngsolve.webgui import Draw
import matplotlib.pyplot as plt
from netgen.geom2d import SplineGeometry
import numpy as np
from ngsolve.nonlinearsolvers import Newton

We want to solve the PDE-constrained shape optimization problem

$$
\min_{\Omega \subset \mathsf{D}} J(u)
:= \int_\Omega |\nabla u_\Omega - \nabla u_{ref}|^2 \, dx
$$

subject to $(\Omega,u_\Omega)$ satisfying

$$
\int_\Omega \nabla u_\Omega \cdot \nabla v + u_\Omega v \, dx
=
\int_\Omega f v \, dx
\quad \text{for all } v \in H_0^1(\Omega),
$$

where $\Omega \subset \mathbb{R}^2$ and
$u_d, f \in C^1(\mathbb{R}^2)$.


In [16]:

geo = OCCGeometry(Circle((0, 0), 1).Face(), dim=2).GenerateMesh(maxh=0.1)
mesh = Mesh(geo)
# #Draw(mesh)

# optimals shape is k lobed flower domain
 
# r = CoefficientFunction((x**2 + y**2)**0.5)
# theta = CoefficientFunction(atan2(y, x))
 
# # Anzahl der Blätter
# k = 6
# # Amplitude der Blätter
# A = 0.5
 
# ud = CF(1 - r**2 + A*cos(k*theta))  
# f  = CF(5 - r**2) 


a = 0.04
b = 0.06

ud = 1 - ((x-0.01)**2/a**2 + (y-0.01)**2/b**2)

f = (2/a**2 + 2/b**2) + ud
# #f = CF(1)

grad_f = CoefficientFunction( (f.Diff(x), f.Diff(y) ) )
grad_ud = CoefficientFunction( (ud.Diff(x), ud.Diff(y) ) )

Draw(ud, mesh, "target_function")


WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

BaseWebGuiScene

In [17]:
fes = H1(mesh, order=2, dirichlet=".*")
u, v = fes.TnT()
gfu = GridFunction(fes)
scene_gfu = Draw (gfu, mesh, "state")

a = BilinearForm(fes, symmetric=True)
a += grad(u)*grad(v)*dx
a += u*v*dx

fstate = LinearForm(fes)
fstate += f*v*dx

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

In [18]:
def SolveStateEquation():
    rhs = gfu.vec.CreateVector()
    rhs.data = fstate.vec - a.mat * gfu.vec
    update = gfu.vec.CreateVector()
    update.data = a.mat.Inverse(fes.FreeDofs()) * rhs
    gfu.vec.data += update

In [19]:
a.Assemble()
fstate.Assemble()
SolveStateEquation()
scene_gfu.Redraw()

### Adjoint equation

The adjoint p satisfies

$$
\int_\Omega \nabla w \cdot \nabla p + w p \, dx
=
-2 \int_\Omega (\nabla u_\Omega - \nabla u_{ref}) w 
\quad \text{for all } w \in H_0^1(\Omega),
$$

In [20]:
def Cost(u):
    return InnerProduct(grad(u) - grad_ud, grad(u) - grad_ud) * dx

p, w = fes.TnT()
gfp = GridFunction(fes)
scene_gfp = Draw (gfp, mesh, "adjoint")

fadjoint = LinearForm(fes)
#fadjoint += -1*Cost(gfu).Diff(gfu,w)
fadjoint += -2* InnerProduct(grad(gfu) - grad_ud, grad(w)) * dx

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

In [21]:
def SolveAdjointEquation():
    rhs = gfp.vec.CreateVector()
    rhs.data = fadjoint.vec - a.mat.T * gfp.vec
    update = gfp.vec.CreateVector()
    update.data = a.mat.Inverse(fes.FreeDofs()).T * rhs
    gfp.vec.data += update

In [22]:
fadjoint.Assemble()
SolveAdjointEquation()
scene_gfp.Redraw()

The shape derivative of the shape function

$$
\mathcal{J}(\Omega) := \int_\Omega |\nabla u - \nabla u_d|^2 \, dx
$$

at $\Omega$ in direction $X$ is given by

$$
\begin{aligned}
DJ(\Omega)(X)
&=
\int_\Omega (\operatorname{div}(X)\,I - \partial X - \partial X^\top)|\nabla u - \nabla u_{\mathrm{ref}}|^2
- 2 \int_\Omega (\nabla u - \nabla u_{\mathrm{ref}}) \nabla^2 u_{\mathrm{ref}} \cdot X\\
&\quad
+ \int_\Omega \operatorname{div}(X) u p \\
&\quad
+ \int_\Omega
\bigl(\operatorname{div}(X)I - \partial X - \partial X^\top\bigr)
\nabla u \cdot \nabla p \\
&\quad
- \int_\Omega (\nabla f \cdot X)\,p\\
& \quad
- \int_\Omega \operatorname{div}(X)fp .
\end{aligned}
$$

We now assemble this shape derivative in NGSolve as follows:


In [23]:
# # gfs = GridFunction(VEC)
# # gfs.Set((1000, 1000))

# # mesh.SetDeformation(gfs)


# h = specialcf.mesh_size
# # print(Integrate(h, mesh, element_wise=True))

# # #help(Integrate)

# Draw(h, mesh)

In [24]:
def check_mesh_quality(mesh, gfSet=None):
    qualities = []
    try:
        for el in mesh.Elements(VOL):
            if len(el.vertices) != 3:
                continue

            try:
                p1_, p2_, p3_ = [mesh[v].point for v in el.vertices]

                if gfSet:
                    p1 = np.array(gfSet(mesh(p1_)))
                    p2 = np.array(gfSet(mesh(p2_)))
                    p3 = np.array(gfSet(mesh(p3_)))
                else:
                    p1 = np.array(p1_)
                    p2 = np.array(p2_)
                    p3 = np.array(p3_)

                a = np.linalg.norm(p2 - p1)
                b = np.linalg.norm(p3 - p2)
                c = np.linalg.norm(p1 - p3)

                qualities.append(max(a, b, c) / min(a, b, c))

            except Exception as e:
                print(f"Error processing element {el}: {e}")
                continue

        qualities = np.array(qualities)
        print(
            f"Mesh quality metrics (min, max, mean, median): "
            f"{np.min(qualities):.6f}, {np.max(qualities):.6f}, "
            f"{np.mean(qualities):.6f}, {np.median(qualities):.6f}"
        )

        # Warning for poor quality elements
        poor_quality = np.sum(qualities > 2)
        if poor_quality > 0:
            print(f"Warning: {poor_quality} elements with poor quality (>2)")

    
    except:
        pass


In [25]:
def hesse(u):
    return CoefficientFunction(
        (u.Diff(x).Diff(x), u.Diff(x).Diff(y),
         u.Diff(y).Diff(x), u.Diff(y).Diff(y)), 
        dims=(2,2)
    )

#VEC = H1(mesh, order=3, dirichlet=".*", dim=2)
VEC = H1(mesh, order=1, dim = 2)
X, PHI = VEC.TnT()

dJOmega = LinearForm(VEC)
dJOmega += InnerProduct((div(PHI)*(grad(gfu)- grad_ud)), grad(gfu)- grad_ud) * dx
dJOmega += -2* InnerProduct(hesse(ud)*PHI, grad(gfu)- grad_ud) * dx
dJOmega += div(PHI)* gfu *gfp *dx
dJOmega += InnerProduct(div(PHI)*grad(gfu), grad(gfp)) * dx
dJOmega += -InnerProduct(PHI.Deriv()*grad(gfu), grad(gfp)) * dx
dJOmega += -InnerProduct(grad(gfu), PHI.Deriv()*grad(gfp)) * dx
dJOmega += -div(PHI)* f * gfp * dx
dJOmega += -grad_f * PHI*gfp * dx

- Next, we want to find a vector field $X$ which yields a decrease of the objective
  function, i.e. we want to find a vector field $X$ such that

  $$
  d\mathcal{J}(\Omega; X) < 0.
  $$

- This can be achieved by solving an auxiliary boundary value problem of the type

  $$
  \text{Find } X \in H :
  \qquad
  b(X, \Phi) = d\mathcal{J}(\Omega; \Phi)
  \quad \text{for all } \Phi \in H,
  $$

  for a suitable Hilbert space $H$ (e.g. $H = H^1(\Omega)^2$).
  Here, $b(\cdot,\cdot) : H \times H \to \mathbb{R}$ is a positive definite bilinear
  form which we are free to choose. Then $-X$ is a descent direction since

  $$
  d\mathcal{J}(\Omega; -X)
  = - d\mathcal{J}(\Omega; X)
  = - b(X, X)
  < 0.
  $$

- First we start with the bilinear form

  $$
  b(X, Y)
  =
  \int_\Omega \nabla X : \nabla Y + X \cdot Y \, dx .
  $$

- we can also choose X as 
  $$
  \min _{X \in H^1_0(\Omega)} \frac{1}{p} \int _\Omega |\nabla X|^p - DJ(\Omega)(X)
  $$

  since this is a continous functional on $H^1_0(\Omega)'$, it is equivalent to finding $X \in H^1_0(\Omega)$ sucht that, 

  $$
  \int _\Omega |\nabla X | ^{p-2}  \nabla X \cdot \nabla \Phi = DJ(\Omega)(\Phi) \qquad \forall \Phi \in H^1_0(\Omega)
  $$

In [26]:
def pose_deformation_equation(gamma=None, p = None): 
    b = BilinearForm(VEC)
    if not p:
        b += InnerProduct(grad(X),grad(PHI))*dx + InnerProduct(X,PHI)*dx

    if gamma:
        print('Using gamma =', gamma)
        b = BilinearForm(VEC)
        b += (1.0/gamma)*InnerProduct(grad(X),grad(PHI))*dx + (1.0/gamma)*InnerProduct(X,PHI)*dx
        # Gamma regularisierung (korrigiert)
        b += InnerProduct(CF((-X[0].Diff(x)+X[1].Diff(y), X[0].Diff(y)+X[1].Diff(x))),
                                   CF((-PHI[0].Diff(x)+PHI[1].Diff(y), PHI[0].Diff(y)+PHI[1].Diff(x)))) * dx

        print("expression tree\n")
        print(b)
        return b
    
    elif p:
        print('Using p =', p)
        eps = 1E-5
    
        grad_norm = sqrt(InnerProduct(grad(X), grad(X)) + eps**2)
        

        c = BilinearForm(VEC)
        c += (grad_norm**(p-2)) * InnerProduct(grad(X), grad(PHI)) * dx
        c += InnerProduct(X,PHI)*dx
        
        # -DJ(Omega)(PHI)
        c += -InnerProduct((div(PHI)*(grad(gfu)- grad_ud)), grad(gfu)- grad_ud) * dx
        c += 2* InnerProduct(grad(gfu)- grad_ud, hesse(ud)*PHI) * dx
        c += -div(PHI)* gfu *gfp *dx
        c += -InnerProduct(div(PHI)*grad(gfu), grad(gfp)) * dx
        c += InnerProduct(PHI.Deriv()*grad(gfu), grad(gfp)) * dx
        c += InnerProduct(grad(gfu), PHI.Deriv()*grad(gfp)) * dx
        c += div(PHI)* f * gfp * dx
        c += grad_f*PHI*gfp * dx
        
        return c

        
    print("expression tree\n")
    print(b)
    return b
   

# def SolveNewton(printrates=True):
#     for ir in range(10):
#         if printrates: 
#             print("Newton step ", ir)
#         res = gfX.vec.CreateVector()
#         c.Apply(gfX.vec, res)
#         #print(res)     
#         res.data -= dJOmega.vec 
        
#         c.AssembleLinearization(gfX.vec)
#         # print(type(c.mat))
#         # plt.spy(c.mat.ToDense())
#         gfX.vec.data -= c.mat.Inverse(VEC.FreeDofs()) * res

def SolveDeformationEquation(linear=True):
    if linear:
        print("solving linear deformation equation")
        rhs = gfX.vec.CreateVector()
        rhs.data = dJOmega.vec - b.mat * gfX.vec
        update = gfX.vec.CreateVector()
        update.data = b.mat.Inverse(VEC.FreeDofs()) * rhs
        gfX.vec.data += update
        #scene_gfset.Redraw()
   
    elif not linear:
        print("solving nonlinear deformation equation with Newton")
        Newton(c, gfX, maxerr = 1E-8, freedofs=VEC.FreeDofs(), maxit = 1000, printing=True)

        #print("Deformation norm:", Norm(gfX.vec)
        #scene_gfset.Redraw()
        


In [27]:
def optimize_shape(linear = True):
    gfset.Set((0,0))
    mesh.SetDeformation(gfset)
    scene.Redraw()
    a.Assemble()
    fstate.Assemble()
    SolveStateEquation()

    LineSearch = True

    iter_max = 1000
    Jold = Integrate(Cost(gfu), mesh)
    converged = False

    # input("Press enter to start optimization")
    print('Cost at initial design', Integrate (Cost(gfu), mesh))
    for k in range(iter_max):
        mesh.SetDeformation(gfset)
        scene.Redraw()

        print('cost at iteration', k, ': ', Jold)

        a.Assemble()
        fstate.Assemble()
        SolveStateEquation()

        fadjoint.Assemble()
        SolveAdjointEquation()
        if linear:
            b.Assemble()
        #plt.spy(b.mat.ToDense())
        dJOmega.Assemble()
        SolveDeformationEquation(linear=linear)

        scale = 0.01 / Norm(gfX.vec)
        gfsetOld = gfset
        gfset.vec.data -= scale * gfX.vec

        Jnew = Integrate(Cost(gfu), mesh)

        if LineSearch:
            while Jnew > Jold and scale > 1e-12:
                #input('a')
                scale = scale / 2
                print("scale = ", scale)
                #print("scale without normalization factor = ", scale* Norm(gfX.vec))
                if scale <= 1e-12:
                    converged = True
                    break

                gfset.vec.data = gfsetOld.vec - scale * gfX.vec         # scale times -X is descent direction
                mesh.SetDeformation(gfset)                              # applying deformation and solve new state

                a.Assemble()                                           # note that during line search we only have to solve state to check if update yields lower costs              
                fstate.Assemble()
                SolveStateEquation()
                Jnew = Integrate(Cost(gfu), mesh)
                print("New cost during line search: ", Jnew)
        
        print("Optimality condition", np.dot(dJOmega.vec.FV().NumPy(), gfX.vec.FV().NumPy()))

        if converged==True:
            print("No more descent can be found")
            break
        Jold = Jnew

        Redraw(blocking=True)

    print("Norm of gset", Integrate(gfset**2, mesh))

### linear

In [28]:
check_mesh_quality(mesh)
gfX = GridFunction(VEC)
#scene_gfset = Draw(gfX, mesh)
#gfX.Set((x *(1-x)*y*(1-y),0)) 
b = pose_deformation_equation()
#b.Assemble()


# gfset denotes the deformation of the original domain and will be updated during the shape optimization
gfset = GridFunction(VEC)
gfset.Set((0,0))

scene = Draw(gfset,mesh,"gfset")
SetVisualization (deformation=True)



optimize_shape(linear = True)
print("Mesh quality after optimization (min, max): ")
mesh.SetDeformation(gfset)
check_mesh_quality(mesh, gfSet=gfset)

# gfX2 = GridFunction(VEC)
# gfX2.vec.data =
# print("=====")
# print(np.dot(dJOmega.vec.FV().NumPy(), gfX.vec.FV().NumPy()))
# print(Integrate(Cost(gfu), mesh))


Mesh quality metrics (min, max, mean, median): 1.004671, 1.566507, 1.149765, 1.125579
expression tree

on space H1HighOrderFESpace(h1ho)
symmetric   = 0
multilevel  = 1
nonassemble = 0
printelmat = 0
elmatev    = 0
eliminate_internal = 0
eliminate_hidden = 0
keep_internal = 0
store_inner = 0
integrators: 
  Symbolic BFI
  Symbolic BFI



WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

Cost at initial design 248080.62354813796
cost at iteration 0 :  248080.62354813796
solving linear deformation equation
Optimality condition 320097544362.0567
cost at iteration 1 :  246432.6052887279
solving linear deformation equation
Optimality condition 317033901121.71063
cost at iteration 2 :  244886.70890078286
solving linear deformation equation
Optimality condition 313995469888.3557
cost at iteration 3 :  243347.47711589537
solving linear deformation equation
Optimality condition 310982094594.0916
cost at iteration 4 :  241814.90235169456
solving linear deformation equation
Optimality condition 307993619819.53
cost at iteration 5 :  240288.9770339856
solving linear deformation equation
Optimality condition 305029890792.021
cost at iteration 6 :  238769.69359596504
solving linear deformation equation
Optimality condition 302090753383.919
cost at iteration 7 :  237257.04447745095
solving linear deformation equation
Optimality condition 299176054110.8615
cost at iteration 8 :  2357

### non linear with p

In [21]:
check_mesh_quality(mesh)
b = pose_deformation_equation(p=3)

gfX = GridFunction(VEC)
#scene_gfset = Draw(gfX, mesh)
# gfset denotes the deformation of the original domain and will be updated during the shape optimization
gfset = GridFunction(VEC)

gfset.Set((0,0))
scene = Draw(gfset,mesh,"gfset")
SetVisualization (deformation=True)

optimize_shape(linear=True)
check_mesh_quality(mesh, gfSet=gfset)

Mesh quality metrics (min, max, mean, median): 1.004671, 1.566507, 1.149765, 1.125579
Using p = 3


WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

Cost at initial design 248080.6235481377
cost at iteration 0 :  248080.6235481377
solving linear deformation equation
Optimality condition 320097544349.62067
cost at iteration 1 :  246432.6052887316
solving linear deformation equation
Optimality condition 317033901109.3932
cost at iteration 2 :  244886.70890079014
solving linear deformation equation
Optimality condition 313995469876.15466
cost at iteration 3 :  243347.4771159062
solving linear deformation equation
Optimality condition 310982094582.0066
cost at iteration 4 :  241814.9023517095
solving linear deformation equation
Optimality condition 307993619807.55994
cost at iteration 5 :  240288.97703400473
solving linear deformation equation
Optimality condition 305029890780.16516
cost at iteration 6 :  238769.6935959879
solving linear deformation equation
Optimality condition 302090753372.17505
cost at iteration 7 :  237257.04447747627
solving linear deformation equation
Optimality condition 299176054099.2311
cost at iteration 8 :  

### linear with gamma

In [27]:
check_mesh_quality(mesh)
b = pose_deformation_equation(gamma=100000)

gfX = GridFunction(VEC)
#scene_gfset = Draw(gfX, mesh)
# gfset denotes the deformation of the original domain and will be updated during the shape optimization
gfset = GridFunction(VEC)

gfset.Set((0,0))
scene = Draw(gfset,mesh,"gfset")
SetVisualization (deformation=True)

optimize_shape()
check_mesh_quality(mesh, gfSet=gfset)

Mesh quality metrics (min, max, mean, median): 1.004671, 1.566507, 1.149765, 1.125579
Using gamma = 100000
expression tree

on space H1HighOrderFESpace(h1ho)
symmetric   = 0
multilevel  = 1
nonassemble = 0
printelmat = 0
elmatev    = 0
eliminate_internal = 0
eliminate_hidden = 0
keep_internal = 0
store_inner = 0
integrators: 
  Symbolic BFI
  Symbolic BFI
  Symbolic BFI



WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

Cost at initial design 248080.62354813787
cost at iteration 0 :  248080.62354813787
solving linear deformation equation
Optimality condition 3.200975443620569e+16
cost at iteration 1 :  246432.6052887279
solving linear deformation equation
Optimality condition 3.1703390112171116e+16
cost at iteration 2 :  244886.70890078312
solving linear deformation equation
Optimality condition 3.139954698883559e+16
cost at iteration 3 :  243347.47711589534
solving linear deformation equation
Optimality condition 3.1098209459409172e+16
cost at iteration 4 :  241814.9023516947
solving linear deformation equation
Optimality condition 3.0799361981952988e+16
cost at iteration 5 :  240288.97703398543
solving linear deformation equation
Optimality condition 3.050298907920209e+16
cost at iteration 6 :  238769.6935959652
solving linear deformation equation
Optimality condition 3.0209075338391836e+16
cost at iteration 7 :  237257.04447745095
solving linear deformation equation
Optimality condition 2.991760541